# Traffic visualization using `sumolib` and `edge-data`

In [ ]:
from matplotlib.collections import LineCollection
from matplotlib import pyplot as plt
import matplotlib
from lxml import etree

if "SUMO_HOME" in os.environ:
    tools = os.path.join(os.environ["SUMO_HOME"], "tools")
    sys.path.append(tools)
else:
    sys.exit("Please declare the environment variable 'SUMO_HOME'")

import sumolib
from sumolib.visualization import helpers

cmap = matplotlib.cm.get_cmap('winter_r')

n = "../simulation/performance/traci-n"
rr = "../simulation/performance/traci-rr"

# fig, (ax1, ax2) = plt.subplots(2,1)
# plot_heatmap(normal, ax1)
# plot_heatmap(rr, ax2)
# fig.set_figheight(50)

fig, ax = plt.subplots()
plot_heatmap(n, ax)

In [ ]:
def plot_heatmap(scenario, ax):
    net = sumolib.net.readNet(f"{scenario}/network/network.net.xml")

    default_color = [0.95 for i in range(3)]
    colors = {}

    default_width = 1
    widths = {}

    for event, edge in etree.iterparse(f"{scenario}/output/edge-data-full-day.xml", tag="edge"):
        c = float(edge.attrib["entered"])
        if c > 0:
            colors[edge.attrib["id"]] = c

        w = float(edge.attrib["entered"])
        if w > 0:
            widths[edge.attrib["id"]] = w
            pass

    # Normalize colors
    max_color = max(colors.values())
    for edge in colors: 
        colors[edge] = cmap(colors[edge] / max_color)

    # Normalize width
    max_width = max(widths.values())
    for edge in widths:
        n_width = (widths[edge] / max_width) * 2
        if n_width > default_width:
            widths[edge] = n_width
        else:
            widths[edge] = default_width

    shapes = []
    c = []
    w = []
    for e in net.getEdges():
        shapes.append(e.getShape())
        if e.getID() in colors:
            c.append(colors[str(e.getID())])
        else:
            c.append(default_color)
        if e._id in widths:
            w.append(widths[str(e._id)])
        else:
            w.append(default_width)

    line_segments = LineCollection(shapes, linewidths=w, colors=c)
    ax.add_collection(line_segments)
    ax.set_xmargin(0.1)
    ax.set_ymargin(0.1)
    ax.autoscale_view(True, True, True)

    norm = matplotlib.colors.Normalize
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm(vmin=1,vmax=max_color))
    plt.colorbar(sm, ax=ax)